<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-1-NLP/blob/master/TPOTClassifier_Spacy_NLP_whiskey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np


In [0]:
#USING VECTORIZATION BY SPACY NLP AND KERAS SEQUENTIAL NEURAL NETWORK FOR TRAINING ON TRAIN DATA AND MAKING PREDICTIONS
#DATASET: WHISKY RATINGS CLASSIFICATION FROM REVIEWS

In [0]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/bs3537/DS-Unit-4-Sprint-1-NLP/master/module3-document-classification/train.csv')

In [0]:
train['text'] = train['description'].str.strip('\n')

In [0]:
train2 = train.drop(columns=['description', 'id'])

In [0]:
train3 = train2.rename(columns={"ratingCategory": "label"})

In [7]:
train3.head(1)

,label,text
0,1,"Sometimes, when whisky is batched, a few lefto..."


In [8]:
from sklearn.model_selection import train_test_split

df_trn, df_val = train_test_split(train3, stratify = train3['label'], test_size = 0.20, random_state=42)

df_trn.shape, df_val.shape

((3269, 2), (818, 2))

In [9]:
#Vectorize text using spacy
pip install -U spacy[cuda92]


Requirement already up-to-date: spacy[cuda92] in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [10]:
!python3 -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [0]:
doc = nlp("NLP is awesome!")

In [0]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [0]:
X = df_trn['text']
y_train = df_trn['label']
X_train_spacy = get_word_vectors(X)

In [0]:
#Val. data

X_val = df_val['text']
y_val = df_val['label']
X_val_spacy = get_word_vectors(X_val)

In [0]:
#TPOT AUTOML MODEL

In [17]:
pip install tpot

In [0]:
from tpot import TPOTClassifier

In [0]:
tpot = TPOTClassifier(scoring='accuracy', random_state=42, n_jobs=-1, verbosity=3, generations=5, population_size=100, periodic_checkpoint_folder="intermediate_results")    #default populations and generations=100

In [0]:
#Standard scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_spacy)

X_val_scaled = scaler.fit_transform(X_val_spacy)

In [23]:
%time
import timeit
times = []
scores = []
winning_pipes = []

# run 2 iterations
for x in range(1):
    start_time = timeit.default_timer()
    tpot.fit(X_train_scaled, y_train)
    elapsed = timeit.default_timer() - start_time
    times.append(elapsed)
    winning_pipes.append(tpot.fitted_pipeline_)
    scores.append(tpot.score(X_val_scaled, y_val))
    tpot.export('tpot_nlp_whisky.py')

# output results
times = [time/60 for time in times]
print('Times:', times)
print('Scores:', scores)   
print('Winning pipelines:', winning_pipes)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs
31 operators have been imported by TPOT.


Skipped pipeline #29 due to time out. Continuing to the next pipeline.
Skipped pipeline #79 due to time out. Continuing to the next pipeline.
Created new folder to save periodic pipeline: intermediate_results
Saving periodic pipeline from pareto front to intermediate_results/pipeline_gen_1_idx_0_2020.04.07_18-18-33.py
Saving periodic pipeline from pareto front to intermediate_results/pipeline_gen_1_idx_1_2020.04.07_18-18-33.py
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty..
_pre_test decorator: _random_mutation_operator: num_test=0 Negat

In [24]:
print(tpot.fitted_pipeline_)
#Winning pipeline is given below

Pipeline(memory=None,
         steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                ('linearsvc',
                 LinearSVC(C=0.1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=42, tol=0.001,
                           verbose=0))],
         verbose=False)


In [0]:
tpot.export('whisky_tpot_digits_pipeline.py')

In [30]:
tpot.score(X_train_scaled, y_train)

0.7858672376873662

In [0]:
#predict on X_val
y_pred = tpot.predict(X_val_spacy)

In [29]:
from sklearn.metrics import accuracy_score
print("Validation accuracy = ", accuracy_score(y_pred, y_val))

Validation accuracy =  0.7053789731051344
